In [23]:
import process_encoding as pe
df = pe.load_df()
df.head()

C:\3team_2nd\process_encoding.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X.Churn = X.Churn.replace("No", 0)
C:\3team_2nd\process_encoding.py:30: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X.CreditRating = X.CreditRating.replace("7-Lowest", 7)


,Churn,MonthlyRevenue,MonthlyMinutes,TotalRecurringCharge,DirectorAssistedCalls,OverageMinutes,RoamingCalls,PercChangeMinutes,PercChangeRevenues,DroppedCalls,...,Occupation_Homemaker,Occupation_Other,Occupation_Professional,Occupation_Retired,Occupation_Self,Occupation_Student,ServiceArea_Center,ServiceArea_East,ServiceArea_West,ServiceArea_None
0,1,24.00,219.0,22.0,0.25,0.0,0.0,-157.0,-19.0,0.7,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,16.99,10.0,17.0,0.00,0.0,0.0,-4.0,0.0,0.3,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0,38.00,8.0,38.0,0.00,0.0,0.0,-2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0,82.28,1312.0,75.0,1.24,0.0,0.0,157.0,8.1,7.7,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1,17.14,0.0,17.0,0.00,0.0,0.0,0.0,-0.2,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [24]:
df.to_csv(f"dataset/encoded_data.csv", index=False)
# df.ServiceArea.valuecount()

In [25]:
%%writefile dataloader.py
import pandas as pd

def load_dataset():
    # load dataset
    data = pd.read_csv('dataset/encoded_data.csv')
    # 컬럼명 소문자로 변경
    data.columns = data.columns.str.lower()
    # target 컬럼을 y로, 나머지를 X로
    X = data.drop(columns='churn')
    y = data['churn']
    
    return X, y
    
def load_testset():
    data = pd.read_csv('./dataset/cell2cellholdout.csv')
    data.columns = data.columns.str.lower()
    # target 컬럼을 y로, 나머지를 X로
    X = data.drop(columns='churn')
    y = data['churn']

    return X,y

Overwriting dataloader.py


In [26]:
from sklearn.model_selection import train_test_split
from dataloader import load_dataset
from dataloader import load_testset
from sklearn.model_selection import train_test_split

X, y = load_dataset()
X_test, y_test = load_testset()

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, stratify=y, random_state=0)

In [27]:
# from sklearn.tree import DecisionTreeClassifier
# tree = DecisionTreeClassifier(random_state=0)
# tree.fit(X_train, y_train)

In [28]:
# !pip install metrics
# !pip install dataloader
# !pip install lightgbm

In [29]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

models = [
    ("LR", LogisticRegression(random_state=0, max_iter=3000)),
    ("GBM", GradientBoostingClassifier(random_state=0)),
    ("RF", RandomForestClassifier(random_state=0)),
    ("KNN", KNeighborsClassifier()),
    ("XGB", XGBClassifier(random_state=0)),
    ("TREE",DecisionTreeClassifier(random_state=0)),
    ("LGBM", LGBMClassifier(random_state=0))
]

In [30]:
baseline_result_train_dict = {}
baseline_result_valid_dict = {}


for name, model in models:
    print(f"Training {name}...")
    model.fit(X_train, y_train)
    
    pred_valid = model.predict(X_valid)
    pred_proba_valid = model.predict_proba(X_valid)[:, 1]

    accuracy_valid = accuracy_score(y_valid, pred_valid)
    precision_valid = precision_score(y_valid, pred_valid)
    recall_valid = recall_score(y_valid, pred_valid)
    f1_valid = f1_score(y_valid, pred_valid)
    auc_valid = roc_auc_score(y_valid, pred_proba_valid)
    
    baseline_result_valid_dict[name] = [accuracy_valid, precision_valid, recall_valid, f1_valid, auc_valid]

Training LR...


C:\Users\Playdata\miniconda3\envs\ml\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Training GBM...
Training RF...
Training KNN...
Training XGB...
Training TREE...
Training LGBM...
[LightGBM] [Info] Number of positive: 10868, number of negative: 27123
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.025887 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3292
[LightGBM] [Info] Number of data points in the train set: 37991, number of used features: 56
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.286068 -> initscore=-0.914559
[LightGBM] [Info] Start training from score -0.914559


In [18]:
import pandas as pd
pd.DataFrame(baseline_result_valid_dict, index=['accuracy', 'precision', 'recall', 'f1', 'roc-auc'])

,LR,GBM,RF,KNN,XGB,TREE,LGBM
accuracy,0.713282,0.718651,0.718730,0.671036,0.709886,0.616077,0.721968
precision,0.453488,0.577720,0.566449,0.358520,0.482205,0.337359,0.571831
recall,0.010765,0.061551,0.071764,0.189898,0.190726,0.354678,0.112062
f1,0.021030,0.111250,0.127389,0.248286,0.273339,0.345802,0.187399
roc-auc,0.614056,0.666818,0.653254,0.558285,0.644684,0.537753,0.670273


In [19]:
import joblib
import os

os.makedirs('models', exist_ok=True)

In [20]:
from sklearn.model_selection import RandomizedSearchCV

param_gbm = {
    'learning_rate': [0.001, 0.01, 0.1, 0.2],
    'n_estimators': [1000, 2000, 3000, 4000, 5000],
    'max_depth': range(1, 6),
    'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1]
}

param_xgb = {
    'learning_rate': [0.001, 0.01, 0.1, 0.2],
    'n_estimators': [1000, 2000, 3000, 4000, 5000],
    'max_depth': range(1, 6),
    'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1]
}

# Initialize RandomizedSearchCV for each model
random_search_gbm = RandomizedSearchCV(GradientBoostingClassifier(random_state=0), param_gbm, cv=4, scoring='roc_auc', n_jobs=-1, n_iter=60, random_state=0)
random_search_xgb = RandomizedSearchCV(XGBClassifier(random_state=0), param_xgb, cv=4, scoring='roc_auc', n_jobs=-1, n_iter=60, random_state=0)

print("GradientBoostingClassifier 튜닝...")
random_search_gbm.fit(X_train, y_train)

print("XGBClassifier 튜닝...")
random_search_xgb.fit(X_train, y_train);

GradientBoostingClassifier 튜닝...
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Playdata\miniconda3\envs\ml\Lib\site-packages\IPython\core\interactiveshell.py", line 3577, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Playdata\AppData\Local\Temp\ipykernel_17560\2276646693.py", line 22, in <module>
    random_search_gbm.fit(X_train, y_train)
  File "C:\Users\Playdata\miniconda3\envs\ml\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Playdata\miniconda3\envs\ml\Lib\site-packages\sklearn\model_selection\_search.py", line 1019, in fit
    self._run_search(evaluate_candidates)
  File "C:\Users\Playdata\miniconda3\envs\ml\Lib\site-packages\sklearn\model_selection\_search.py", line 1960, in _run_search
    evaluate_candidates(
  File "C:\Users\Playdata\miniconda3\envs\ml\Lib\site-packages\sklearn\model_selection\_search.py", line 965, in evaluate_candidates
 

In [ ]:
## Best parameter 확인
print("GradientBoostingClassifier Best parameters:", random_search_gbm.best_params_)
print("XGBClassifier Best parameters:", random_search_xgb.best_params_)

In [ ]:
print("GradientBoostingClassifier Best score:", random_search_gbm.best_score_)
print("XGBClassifier Best score:", random_search_xgb.best_score_)

In [ ]:
best_gbm = random_search_gbm.best_estimator_
best_xgb = random_search_xgb.best_estimator_

In [ ]:
joblib.dump(best_gbm, 'models/best_gbm.pkl')
joblib.dump(best_xgb, 'models/best_xgb.pkl')

print("Models saved successfully.")

In [ ]:
best_models = [
    ("GradientBoostingClassifier", best_gbm),
    ("XGBClassifier", best_xgb)
]

test_results = {}

for name, model in best_models:
    print(f"평가 {name}...")
    pred_test = model.predict(X_test_preprocessed)
    pred_proba_test = model.predict_proba(X_test_preprocessed)[:, 1]

    accuracy_test = accuracy_score(y_test, pred_test)
    precision_test = precision_score(y_test, pred_test)
    recall_test = recall_score(y_test, pred_test)
    f1_test = f1_score(y_test, pred_test)
    auc_test = roc_auc_score(y_test, pred_proba_test)
    
    test_results[name] = [accuracy_test, precision_test, recall_test, f1_test, auc_test]

In [ ]:
pd.DataFrame(test_results, index=['accuracy', 'precision', 'recall', 'f1', 'roc-auc'])

In [ ]:
#LogisticRegression을 통한 Feature Importance 시각화 
import matplotlib.pyplot as plt
import numpy as np

model_lr = LogisticRegression(random_state=0, max_iter=3000)

model_lr.fit(X_train, y_train)

# coefficient

coefficients = model_lr.coef_[0]
indices = np.argsort(np.abs(coefficients))  
plt.figure(figsize=(10, 6))
plt.title("Feature Importance (LR)")
plt.barh(range(len(indices)), coefficients[indices], align="center")
plt.yticks(range(len(indices)), [X_train.columns[i] for i in indices])
plt.xlabel("Coefficient Value")
plt.show()